# **Contains all the necessary scripts for static website development**

### **Importing libraries**

In [2]:
import yaml
import pandas as pd

## **Script for generating tables from data-catalogs**

- Two separate tables one for ONS datasets and for Other datasets
- Extracting name, description, zip-url and website-url from the catalog and making a dataframe
- Converting the Dataframe to html to populate in the markdown file (.md)

In [32]:
#Read the data-catalog
with open('../data_catalog.yaml','r') as file:
    catalog = yaml.safe_load(file)

In [33]:
other_data = []

for item in catalog['single_data_sources']:
    name = item.get('name')
    description = item.get('description')
    zip_url = item.get('zip_url')
    source_website = item.get('website_url')
    other_data.append([name,description,zip_url,source_website])
    
other_df = pd.DataFrame(other_data, columns=['Name','Description','Zip-URL','Source website'])

# Converting zip-url and website-url to links in html table
other_df['Zip-URL'] = other_df['Zip-URL'].apply(lambda x: f'<a href="{x}">Download Zip file</a>')
other_df['Source website'] = other_df['Source website'].apply(lambda x: f'<a href="{x}">View data source</a>')
other_df

,Name,Description,Zip-URL,Source website
0,NHS GP Appointments by Region,NHS England / digital published version of GP ...,"<a href=""https://files.digital.nhs.uk/3D/ED1ED...","<a href=""https://digital.nhs.uk/data-and-infor..."
1,2023-24 Monthly Outpatient Referrals Data,The Monthly Referral Return (MRR) contains dat...,"<a href=""https://www.england.nhs.uk/statistics...","<a href=""https://www.england.nhs.uk/statistics..."
2,2022-23 Monthly Outpatient Referrals Data,The Monthly Referral Return (MRR) contains dat...,"<a href=""https://www.england.nhs.uk/statistics...","<a href=""https://www.england.nhs.uk/statistics..."
3,2021-22 Monthly Outpatient Referrals Data,The Monthly Referral Return (MRR) contains dat...,"<a href=""https://www.england.nhs.uk/statistics...","<a href=""https://www.england.nhs.uk/statistics..."
4,General Practice workforce August 2023,The General Practice Workforce series of Offic...,"<a href=""https://files.digital.nhs.uk/67/A6C43...","<a href=""https://digital.nhs.uk/data-and-infor..."
5,"Appointments in General Practice, September 2023",The publication includes important information...,"<a href=""https://files.digital.nhs.uk/87/A548F...","<a href=""https://digital.nhs.uk/data-and-infor..."
6,"Patients Registered at a GP practice, October ...",The publication contains data for number of pa...,"<a href=""https://files.digital.nhs.uk/4B/EA9F9...","<a href=""https://digital.nhs.uk/data-and-infor..."


In [34]:
scenarios_data_source = catalog['scenario_data_sources']
scenarios = scenarios_data_source[0]['scenarios']

source_website_ons = scenarios_data_source[0]['website_url']

ONS_data = []

for item in scenarios:
    name = item.get('name')
    description = item.get('description')
    zip_url = item.get('zip_url')
    ONS_data.append([name,description,zip_url,source_website_ons])
    
ons_df = pd.DataFrame(ONS_data, columns=['Name','Description','Zip-URL','Source website (ONS)'])

# Converting zip-url and website-url to links in html table
ons_df['Zip-URL'] = ons_df['Zip-URL'].apply(lambda x: f'<a href="{x}">Download Zip file</a>')
ons_df['Source website (ONS)'] = ons_df['Source website (ONS)'].apply(lambda x: f'<a href="{x}">View data source</a>')
ons_df

,Name,Description,Zip-URL,Source website (ONS)
0,Principal projection,"Population figures over a 25-year period, by s...","<a href=""https://www.ons.gov.uk/file?uri=/peop...","<a href=""https://www.ons.gov.uk/peoplepopulati..."
1,10 year migration variant,The 10-year migration variant was created beca...,"<a href=""https://www.ons.gov.uk/file?uri=/peop...","<a href=""https://www.ons.gov.uk/peoplepopulati..."
2,High international migration variant,The high and low international migration varia...,"<a href=""https://www.ons.gov.uk/file?uri=/peop...","<a href=""https://www.ons.gov.uk/peoplepopulati..."
3,Low international migration variant,The high and low international migration varia...,"<a href=""https://www.ons.gov.uk/file?uri=/peop...","<a href=""https://www.ons.gov.uk/peoplepopulati..."
4,Alternative internal migration variant,The high and low international migration varia...,"<a href=""https://www.ons.gov.uk/file?uri=/peop...","<a href=""https://www.ons.gov.uk/peoplepopulati..."


In [68]:
# Converting dataframes to HTML table with a Caption
html_table_others = f'<table><caption>Other Datasets</caption>{other_df.to_html(index=False, escape=False)}'
html_table_ons = f'<table><caption>ONS Datasets</caption>{ons_df.to_html(index=False, escape=False)}'

#Reading the markdown file where we need to push the table
with open('/workspaces/PrimaryCareDemandAndCapacity/staticWebsite/content/pages/Catalogs.md', 'r') as md_file:
    lines = md_file.readlines()
    
start_index1 = -1
end_index1 = -1
for i, line in enumerate(lines):
    if '<!-- START TABLE 1-->' in line:
        start_index1 = i
    elif '<!-- END TABLE 1-->' in line:
        end_index1 = i

start_index2 = -1
end_index2 = -1
for i, line in enumerate(lines):
    if '<!-- START TABLE 2-->' in line:
        start_index2 = i
    elif '<!-- END TABLE 2-->' in line:
        end_index2 = i

# Keeping the lines before table 1
updated_content = lines[:start_index1 +1]

# Appending table 1
updated_content.append(html_table_others)

# Keeping the lines between table 1 and table 2
updated_content.extend(lines[end_index1:start_index2 +1])

# Appending table 2
updated_content.append(html_table_ons)

#Keeping the lines after table 2
updated_content.extend(lines[end_index2:])
    
    
# Writing the HTML table to the markdown file
with open('/workspaces/PrimaryCareDemandAndCapacity/staticWebsite/content/pages/Catalogs.md', 'w') as md_file:
    md_file.writelines(updated_content)

In [60]:
updated_content

['Title: Data Catalog\n',
 '\n',
 '<!-- START TABLE 1-->\n',
 '<table><caption>Other Datasets</caption><table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th>Name</th>\n      <th>Description</th>\n      <th>Zip-URL</th>\n      <th>Source website</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <td>NHS GP Appointments by Region</td>\n      <td>NHS England / digital published version of GP appointments, by sub-ICB area.</td>\n      <td><a href="https://files.digital.nhs.uk/3D/ED1EDE/Appointments_GP_Regional_CSV_Aug_23.zip">Download Zip file</a></td>\n      <td><a href="https://digital.nhs.uk/data-and-information/publications/statistical/appointments-in-general-practice">View data source</a></td>\n    </tr>\n    <tr>\n      <td>2023-24 Monthly Outpatient Referrals Data</td>\n      <td>The Monthly Referral Return (MRR) contains data relating to GP and other referrals for first consultant-led outpatient appointments focusing on all treatment f